In [5]:
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
SCREEN_WIDTH = 1470
SCREEN_HEIGHT = 956
HEADLESS = False
MAX_TIME_TO_WAIT = 20

# Cria o driver e ajusta para que a tela do navegador fique do lado esquerdo
options = Options()
options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
if HEADLESS: options.add_argument('--headless')  # Ativa o modo headless
driver = webdriver.Chrome(options=options)
if not HEADLESS:
    driver.set_window_position(0, 0)
    driver.set_window_size(SCREEN_WIDTH // 2, SCREEN_HEIGHT)  # Para ficar do lado esquerdo
driver.get('https://ufabcnext.com/')

# Localiza botão do Facebook e clica
facebook_button = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
)
facebook_button.click()

# Escreve o e-mail e RA
input_email = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.presence_of_element_located((By.ID, 'input-0'))
)
input_email.send_keys(os.environ['EMAIL'])
input_ra = driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
input_ra.send_keys(os.environ['RA'])
submit_button = driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
submit_button.click()

# Procura por uma matéria em específico
subjects_input = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.presence_of_element_located((By.ID, 'input-20'))
)
subjects_input.send_keys('bases matem')
option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'v-list-item')][contains(., 'bases matemáticas')]"))
)
option.click()

# Puxa os valores e imprime na tela
grades_percentage = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
    EC.visibility_of_all_elements_located((By.CLASS_NAME, 'highcharts-text-outline'))
)
grades_dict = {gp[0]: gp[1].split('%')[0].strip() for gp in [gp_.text.split(':') for gp_ in grades_percentage]}

for grade, percentual in grades_dict.items():
    print(f'{grade}: {percentual}%')
    
time.sleep(5)
driver.quit()

A: 8.8%
B: 15.4%
C: 25.8%
D: 15.5%
F: 29.0%
O: 5.5%


In [1]:
import re

text = 'Termodinâmica Aplicada I'
text_lower = text.lower().strip()
text_splitted = re.split(r'[áéíóúãõâêôü]', text)[0]

text_lower, text_splitted

('termodinâmica aplicada i', 'Termodin')

Agora separando:

In [2]:
SCREEN_WIDTH = 1470
SCREEN_HEIGHT = 956
HEADLESS = False
MAX_TIME_TO_WAIT = 20
IS_MAC = True
COOLDOWN_TIME = 2

In [ ]:
from typing import Literal

class UFABCNextGradeScraper:
    def __init__(self, options: Options, credentials: dict[Literal['e-mail', 'ra'], str], max_time_to_wait: float):
        self.__setup_driver(options)
        self.__credentials = credentials
        self.__max_time_to_wait = max_time_to_wait


    def __setup_driver(self, options):
        self.driver = webdriver.Chrome(options=options)


    def login(self):
        # Acessa a página do UFABC Next
        self.driver.get('https://ufabcnext.com')

        # Localiza botão do Facebook e clica
        facebook_button = WebDriverWait(self.driver, self.max_time_to_wait).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
        )
        facebook_button.click()

        # Escreve o e-mail , RA e depois submete para fazer o login
        input_email = WebDriverWait(self.driver, self.max_time_to_wait).until(
            EC.presence_of_element_located((By.ID, 'input-0'))
        )
        input_email.send_keys(os.environ['EMAIL'])
        input_ra = self.driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
        input_ra.send_keys(os.environ['RA'])
        submit_button = self.driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
        submit_button.click()


    


In [ ]:
def set_driver():
    # Cria o driver e ajusta para que a tela do navegador fique do lado esquerdo
    options = Options()
    options.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
    if HEADLESS: options.add_argument('--headless')  # Ativa o modo headless
    driver = webdriver.Chrome(options=options)
    if not HEADLESS:
        driver.set_window_position(0, 0)
        driver.set_window_size(SCREEN_WIDTH // 2, SCREEN_HEIGHT)  # Para ficar do lado esquerdo

    return driver

def login(driver):
    driver.get('https://ufabcnext.com/')

    # Localiza botão do Facebook e clica
    facebook_button = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
    )
    facebook_button.click()

    # Escreve o e-mail e RA
    input_email = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
        EC.presence_of_element_located((By.ID, 'input-0'))
    )
    input_email.send_keys(os.environ['EMAIL'])
    input_ra = driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
    input_ra.send_keys(os.environ['RA'])
    submit_button = driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
    submit_button.click()

    return driver

def get_grades_dict(driver, subject_name, verbose=False):
    subject_name_std, subject_name_splitted = standardize_name(subject_name)

    # Procura por uma matéria em específico
    subjects_input = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
        EC.presence_of_element_located((By.ID, 'input-20'))
    )
    if IS_MAC: 
        subjects_input.send_keys(Keys.COMMAND + 'a')  # Seleciona tudo (COMMAND pois é Mac)
    else:
        subjects_input.send_keys(Keys.CONTROL + 'a')
    subjects_input.send_keys(Keys.DELETE)  # Apaga o que acabou de selecionar
    subjects_input.send_keys(subject_name_splitted)
    option = WebDriverWait(driver, 10).until(
        # EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][contains(., '{subject_name_std}')]"))  # Se o texto conter aquilo
        EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][text()=' {subject_name_std}']"))  # Se o texto for igual! Tem sempre um espacinho na frente. Pode ser perigoso se houver uma pequena diferença!
    )
    option.click()

    # Puxa os valores e imprime na tela
    grades_percentage = WebDriverWait(driver, MAX_TIME_TO_WAIT).until(
        EC.visibility_of_all_elements_located((By.CLASS_NAME, 'highcharts-text-outline'))
    )
    grades_dict = {gp[0]: gp[1].split('%')[0].strip() for gp in [gp_.text.split(':') for gp_ in grades_percentage]}

    if verbose:
        for grade, percentual in grades_dict.items():
            print(f'{grade}: {percentual}%')

    return driver, grades_dict


def standardize_name(name):
    name_std = name.lower().strip()
    name_splitted_list = re.split(r'[áéíóúàãõâêôüàç,.:/|]', name_std)
    name_splitted = sorted(name_splitted_list, key=lambda s: -len(s))[0]  # Pega a maior parte

    return name_std, name_splitted

In [6]:

import traceback

subjects = ['Bases Matemáticas', 'Termodinâmica Aplicada I', 'Sistemas CAD/CAM']

driver = set_driver()
driver = login(driver)

subjects_dict = {}
for s in subjects:
    try:
        print(f'{s.upper()}:\n')
        driver, sd = get_grades_dict(driver, subject_name=s, verbose=True)
        subjects_dict[s] = sd

    except Exception:
        print('ERRO DURANTE A EXECUÇÃO DE BUSCA!')
        traceback.print_exc()

    finally:
        print(75*'-', end='\n')
        time.sleep(COOLDOWN_TIME)  # Para evitar de tomar cooldown

# driver.quit()
driver.quit()

BASES MATEMÁTICAS:

A: 8.8%
B: 15.4%
C: 25.9%
D: 15.5%
F: 29.0%
O: 5.5%
---------------------------------------------------------------------------
TERMODINÂMICA APLICADA I:

A: 9.7%
B: 14.6%
C: 24.6%
D: 10.6%
F: 30.4%
O: 10.1%
---------------------------------------------------------------------------
SISTEMAS CAD/CAM:

A: 18.6%
B: 33.4%
C: 28.3%
D: 9.5%
F: 4.0%
O: 6.1%
---------------------------------------------------------------------------


In [132]:
import traceback
import pandas as pd

df_grades_ufabc = pd.read_excel('./data/notas-ufabc.xlsx')
subjects = df_grades_ufabc['DISCIPLINA'].values

driver = set_driver()
driver = login(driver)

subjects_dict = {}
error_subjects = []
for s in subjects:
    try:
        print(f'{s.upper()}:\n')
        driver, sd = get_grades_dict(driver, subject_name=s, verbose=True)
        subjects_dict[s] = sd

    except Exception:
        print('ERRO DURANTE A EXECUÇÃO DE BUSCA!')
        # traceback.print_exc()
        error_subjects.append(s)

    finally:
        print(75*'-', end='\n')
        time.sleep(COOLDOWN_TIME)  # Para evitar de tomar cooldown

# driver.quit()

BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS:

A: 69.6%
B: 25.1%
C: 3.8%
D: 0.4%
F: 0.2%
O: 0.9%
---------------------------------------------------------------------------
BASES COMPUTACIONAIS DA CIÊNCIA:

A: 39.1%
B: 26.9%
C: 18.3%
D: 7.3%
F: 6.4%
O: 2.0%
---------------------------------------------------------------------------
BASES CONCEITUAIS DA ENERGIA:

A: 25.3%
B: 33.7%
C: 24.4%
D: 7.2%
F: 7.9%
O: 1.4%
---------------------------------------------------------------------------
BASES MATEMÁTICAS:

A: 8.8%
B: 15.4%
C: 25.8%
D: 15.5%
F: 29.0%
O: 5.5%
---------------------------------------------------------------------------
ESTRUTURA DA MATÉRIA:

A: 17.5%
B: 29.5%
C: 31.4%
D: 10.4%
F: 8.9%
O: 2.4%
---------------------------------------------------------------------------
EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA:

A: 31.0%
B: 43.1%
C: 17.4%
D: 4.8%
F: 2.3%
O: 1.4%
---------------------------------------------------------------------------
BIODIVERSIDADE: INTERAÇÕES ENTRE ORGANISMO

In [133]:
error_subjects

['FENÔMENOS MECÂNICOS', 'FOTÔNICA', 'INSTRUMENTAÇÃO E METROLOGIA ÓPTICA']

In [163]:
import numpy as np

df_tmp = pd.DataFrame.from_dict(subjects_dict).T.reset_index().rename(columns={'index': 'Matéria'}).fillna(0)
df_tmp['F'] = df_tmp['F'].astype(float) + df_tmp['O'].astype(float)
df_tmp = df_tmp.drop(columns=['O'])
df_tmp[['A', 'B', 'C', 'D', 'F']] = df_tmp[['A', 'B', 'C', 'D', 'F']].astype(float)
df_tmp['Nota provável'] = df_tmp[['A', 'B', 'C', 'D', 'F']].apply(lambda s: (s['A']*4 + s['B']*3 + s['C']*2 + s['D']*1)/s[['A', 'B', 'C', 'D', 'F']].sum(), axis=1)
df_tmp['Conceito provável'] = df_tmp['Nota provável'].apply(lambda s: 'F' if 0.0 <= s < 0.8 
                                                                      else 'D' if 0.8 <= s < 1.6
                                                                      else 'C' if 1.6 <= s < 2.4
                                                                      else 'B' if 2.4 <= s < 3.2
                                                                      else 'A')
df_tmp['Conceito moda'] = df_tmp[['A', 'B', 'C', 'D', 'F']].apply(lambda s: np.argmax(s), axis=1).map({0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'})
df_tmp

,Matéria,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
0,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,69.6,25.1,3.8,0.4,1.1,3.617000,A,A
1,BASES COMPUTACIONAIS DA CIÊNCIA,39.1,26.9,18.3,7.3,8.4,2.810000,B,A
2,BASES CONCEITUAIS DA ENERGIA,25.3,33.7,24.4,7.2,9.3,2.585586,B,B
3,BASES MATEMÁTICAS,8.8,15.4,25.8,15.5,34.5,1.485000,D,F
4,ESTRUTURA DA MATÉRIA,17.5,29.5,31.4,10.4,11.3,2.314685,C,C
...,...,...,...,...,...,...,...,...,...
73,MECÂNICA DOS FLUIDOS I,5.9,14.0,26.3,13.4,40.4,1.316000,D,F
74,TRABALHO DE GRADUAÇÃO II EM ENGENHARIA DE INST...,62.5,16.7,4.2,0.0,16.7,3.081918,B,A
75,SISTEMAS DE CONTROLE II,12.1,21.5,24.3,15.5,26.6,1.770000,C,F
76,TEORIA DE ACIONAMENTOS ELÉTRICOS,25.4,29.4,22.7,10.2,12.4,2.451548,B,B


In [165]:
df_tmp['Conceito provável'].value_counts(normalize=True)

Conceito provável
C    0.474359
B    0.371795
A    0.102564
D    0.051282
Name: proportion, dtype: float64

In [173]:
df_tmp.sort_values(by='F', ascending=False)

,Matéria,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
37,TERMODINÂMICA APLICADA I,9.7,14.6,24.6,10.6,40.5,1.424000,D,F
73,MECÂNICA DOS FLUIDOS I,5.9,14.0,26.3,13.4,40.4,1.316000,D,F
7,FUNÇÕES DE UMA VARIÁVEL,11.7,14.4,22.1,11.9,40.0,1.459540,D,F
3,BASES MATEMÁTICAS,8.8,15.4,25.8,15.5,34.5,1.485000,D,F
64,CONFIABILIDADE DE COMPONENTES E SISTEMAS,23.4,15.5,12.9,14.6,33.7,1.803197,C,F
...,...,...,...,...,...,...,...,...,...
59,ENGENHARIA UNIFICADA II,65.6,24.0,6.7,0.3,3.4,3.481000,A,A
52,ENGENHARIA UNIFICADA I,63.9,26.1,6.0,0.6,3.3,3.468468,A,A
27,INTRODUÇÃO ÀS ENGENHARIAS,59.2,29.8,7.0,1.0,2.9,3.415415,A,A
0,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,69.6,25.1,3.8,0.4,1.1,3.617000,A,A


In [166]:
df_tmp['Conceito moda'].value_counts(normalize=True)

Conceito moda
A    0.435897
F    0.217949
B    0.179487
C    0.166667
Name: proportion, dtype: float64

In [171]:
df_felipe = pd.read_excel('./data/notas-ufabc.xlsx')
df_felipe[df_felipe['RESULTADO'] != 'E']['RESULTADO'].value_counts(normalize=True)

RESULTADO
A    0.717949
B    0.243590
C    0.038462
Name: proportion, dtype: float64

In [167]:
df_tmp[df_tmp['Conceito moda'] == 'A']

,Matéria,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
0,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,69.6,25.1,3.8,0.4,1.1,3.617000,A,A
1,BASES COMPUTACIONAIS DA CIÊNCIA,39.1,26.9,18.3,7.3,8.4,2.810000,B,A
6,BIODIVERSIDADE: INTERAÇÕES ENTRE ORGANISMOS E ...,41.0,34.1,17.0,2.9,5.0,3.032000,B,A
11,PROCESSAMENTO DA INFORMAÇÃO,29.2,24.2,20.0,10.1,16.5,2.395000,C,A
12,TRANSFORMAÇÕES QUÍMICAS,29.9,27.1,22.7,10.8,9.5,2.571000,B,A
21,BASES EPISTEMOLÓGICAS DA CIÊNCIA MODERNA,32.1,30.1,19.7,8.2,9.9,2.663000,B,A
23,FUNDAMENTOS DE DESENHO TÉCNICO,37.1,29.3,17.5,7.6,8.5,2.789000,B,A
26,ESTRUTURA E DINÂMICA SOCIAL,38.3,35.3,17.1,4.6,4.7,2.979000,B,A
27,INTRODUÇÃO ÀS ENGENHARIAS,59.2,29.8,7.0,1.0,2.9,3.415415,A,A
29,PRINCÍPIOS DE ADMINISTRAÇÃO,42.1,32.9,16.9,3.7,4.5,3.042957,B,A
